In [1]:
import pandas as pd
import metapredict as meta
import numpy as np

from __future__ import print_function
import re, gzip, time, itertools, io
import sys
import pandas as pd
import numpy as np
#import requests
import matplotlib.pyplot as plt
import Bio
from Bio import SeqIO
from Bio.SeqIO.FastaIO import SimpleFastaParser
import json


In [ ]:
##first run metapredict
#../HG38_pep/HG38_pep_unique_onlyids.fasta -o HG38_FULL.IDR_pLDDT.IDR_New.fasta --threshold 0.5 --minIDR 20 --gapclosure 5 --minFold 20

In [30]:
data= pd.read_csv("../HG38_pep/HG30_pep_annotation.csv")


######## regex expression
p2 = re.compile(r'(?P<ID>[^\s]+)+'
                   + r'.IDR_START=(?P<IDR_START>\d{1,5})'
                   + r'.IDR_END=(?P<IDR_END>\d{1,5})')

####### open metapredict fasta file and make table out of it

with open('HG38_FULL.IDR_pLDDT.IDR_New.fasta') as fasta_file:  # Will close handle cleanly
    identifiers = []
    #IDRseq = []
    IDRlengths = []
    for title, sequence in SimpleFastaParser(fasta_file):
        identifiers.append(title)  # First word is ID
        IDRlengths.append(len(sequence))
        #IDRseq.append(sequence)
s1 = pd.Series(identifiers, name='header')
s2 = pd.Series(IDRlengths, name='IDRlengths')
#s3 = pd.Series(IDRseq, name = 'IDRseq')
HG38_FULL_pep_IDR = pd.DataFrame(dict(header=s1))
HG38_FULL_pep_IDR = pd.concat([
    HG38_FULL_pep_IDR,
    (
        HG38_FULL_pep_IDR.header.str.extractall(p2)
          .reset_index('match', drop=True)
    )
], axis=1).fillna('')
HG38_FULL_pep_IDR = HG38_FULL_pep_IDR.drop('header', 1)
HG38_FULL_pep_IDR["IDRlengths"] = s2
#HG38IDR["IDRseq"] = s3
HG38_FULL_pep_IDR.head()



,ID,IDR_START,IDR_END,IDRlengths
0,ENSP00000250823.4,0,125,125
1,ENSP00000342812.3,0,68,68
2,ENSP00000342812.3,972,996,24
3,ENSP00000342812.3,2474,2555,81
4,ENSP00000354722.2,0,24,24


In [31]:
df=HG38_FULL_pep_IDR.drop(['IDRlengths'], axis=1)

In [32]:
df['index']=df['IDR_START'].astype(str)+" "+df['IDR_END'].astype(str)

In [34]:
df=df.drop(['IDR_START'], axis=1)
df=df.drop(['IDR_END'], axis=1)

In [36]:
df_new=df.groupby('ID').apply(lambda dfg: dfg.drop('ID', axis=1).to_dict(orient='list')).to_dict()

In [75]:
indexesforkeys={}

for i in df_new.keys():
    aa=(list((df_new[i]).values())[0])
    ab= [i.split(" ", 1) for i in aa]
    flat_list = [item for sublist in ab for item in sublist]
    indexesforkeys[i]=list(map(int, flat_list))



In [79]:
##save metapredict IDR indexes
with open("../metapredict/metapredict_least20_concIDRs.json", "w") as fp:
    json.dump(indexesforkeys , fp) 